In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as opt

In [ ]:
df = pd.read_csv('dataset/AGE_PREDICTION.csv')
df.head()

In [ ]:
def mse_loss(y_real, y_pred, alpha, weights):
    reg = alpha*(weights**2)

    loss = np.mean((y_real-y_pred)**2)
    return loss+reg

,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,...,feat_24,feat_25,feat_26,feat_27,feat_28,feat_29,feat_30,feat_31,feat_32,gt
0,2.686191,-0.989465,-0.920503,1.607427,-0.896248,1.118974,-0.969456,1.811707,2.560955,3.803463,...,-0.862891,-0.909545,-0.915361,-0.952061,-0.989461,1.911855,1.409705,2.303997,-0.981840,54
1,-0.887917,4.915272,-0.939446,-0.343677,-0.964685,-0.478649,4.342395,-0.332870,-0.768041,-0.815375,...,-0.939201,-0.965917,-0.969461,-0.934799,5.304822,0.934790,-0.410701,0.284690,4.919212,18
2,-0.923215,2.746968,-0.918085,0.047804,-0.908587,-0.451752,2.984481,0.535007,-0.591029,-0.324043,...,-0.809726,-0.929934,-0.891814,-0.881796,3.415373,1.044108,-0.442615,0.033648,2.628199,26
3,-0.268866,-0.408416,-0.935145,0.731800,-0.922438,0.221781,-0.046606,1.149634,0.592136,1.357959,...,-0.834968,-0.937475,-0.917737,-0.929519,-0.226282,1.608048,0.276169,1.246468,-0.363367,33
4,0.529231,-0.829957,-0.897425,0.921280,-0.865304,0.331018,-0.644940,1.296097,1.166863,2.036034,...,-0.775411,-0.881967,-0.864018,-0.908001,-0.784495,1.329586,0.547925,1.195395,-0.810089,35


In [ ]:
class NeuralNetwork:
    def _init_(self, neurons, activation='tanh'):
        """
        neurons: list with the number of neurons in each layer.
                      e.g. [512, 16, 8, 1]
        activation: string, 'tanh' or 'sigmoid'
        """
        self.neurons = neurons
        self.layers = len(neurons) - 1  # number of layers (excluding input)
        self.activation = activation

        # Initialize weights and biases
        self.weights = []
        self.biases = []
        for i in range(self.layers):
            in_dim = neurons[i]
            out_dim = neurons[i + 1]
            W = np.random.randn(in_dim, out_dim) * 0.01
            b = np.zeros(out_dim)
            self.weights.append(W)
            self.biases.append(b)

    def activation(self, x):
        if self.activation == 'tanh':
            return np.tanh(x)
        elif self.activation == 'sigmoid':
            return 1 / (1 + np.exp(-x))
        else:
            raise ValueError("Unsupported activation function")
        
    def derivate_activation(self, x):
        if self.activation == 'tanh':
            return 1-self.activation(x)
        elif self.activation == 'sigmoid':
            return self.activation(x)*(1-self.activation(x))

    def forward(self, X):
        """
        Performs a forward pass through the network.
        X: input data of shape (n_samples, n_features)
        Returns:
            Output prediction of shape (n_samples, 1)
        """
        a = X
        for i in range(self.layers - 1):  # all hidden layers
            z = np.dot(a, self.weights[i]) + self.biases[i]
            a = self.activation(z)
        # Output layer (no activation)
        output = np.dot(a, self.weights[-1]) + self.biases[-1]
        return output

    def get_params_vector(self):
        """Returns all weights and biases flattened into a single vector."""
        params = []
        for W, b in zip(self.weights, self.biases):
            params.append(W.flatten())
            params.append(b.flatten())
        return np.concatenate(params)

    def set_params_vector(self, flat_params):
        """Set the weights and biases from a flat parameter vector."""
        idx = 0
        self.weights = []
        self.biases = []
        for i in range(self.layers):
            in_dim = self.neurons[i]
            out_dim = self.neurons[i + 1]
            w_size = in_dim * out_dim
            b_size = out_dim

            W = flat_params[idx:idx + w_size].reshape((in_dim, out_dim))
            idx += w_size
            b = flat_params[idx:idx + b_size]
            idx += b_size

            self.weights.append(W)
            self.biases.append(b)

    def mse_loss(self, y_real, y_pred, alpha, weights):
        """
        y_real: true target values
        y_pred: predicted values
        alpha: regularization parameter
        weights: list of weight matrices
        """
        loss = np.mean((y_real - y_pred)**2)
        reg = sum([np.sum(w**2) for w in weights])
        return loss + alpha * reg
    
    def loss(flat_params, model, X, y, alpha):
        """
        flat_params: flat vector of weights and biases
        model: NeuralNetwork model
        X: input features
        y: target values
        alpha: regularization parameter
        """
        model.set_params_vector(flat_params)
        y_pred = model.forward(X)
        return mse_loss(y, y_pred, alpha, model.weights)
    
    def mape(y_real, y_pred):
        """
        y_real: true target values
        y_pred: predicted values
        """
        return np.sum(np.abs((y_real-y_pred)/y_real))*(100/len(y_real))